In [1]:
# Instalacija PySwarms biblioteke za optimizaciju roja čestica
!pip install pyswarms

# Instalacija TensorFlow biblioteke. Keras je njen sastavni dio i potreban nam je
# za funkciju gubitka (CategoricalCrossentropy).
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.3 MB/s eta 0:00:00


In [2]:
# Osnovne biblioteke
import numpy as np
import pyswarms as ps

# Moduli iz Scikit-learn biblioteke za učitavanje Iris dataseta
# i podjelu podataka na trening i testni skup
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Modul iz Keras (TensorFlow) biblioteke za funkciju gubitka
from tensorflow.keras import losses

# Podešavanje da NumPy ne ispisuje brojeve u scientific notaciji radi lakšeg čitanja
np.set_printoptions(suppress=True)

2025-06-13 20:03:40,129 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


In [3]:
# Učitavanje Iris dataseta
data = load_iris()

# Odvajanje značajki (X) i labela (y)
X = data.data
y_original = data.target

# Kreiranje nove matrice za one-hot enkodirane labele
y = np.zeros((y_original.shape[0], 3))
for i in range(y_original.shape[0]):
    y[i, y_original[i]] = 1

# Podjela podataka na trening (70%) i testni (30%) skup
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Dimenzije trening skupa (značajke):", X_train.shape)
print("Dimenzije trening skupa (labele):", y_train.shape)
print("Dimenzije testnog skupa (značajke):", X_test.shape)
print("Dimenzije testnog skupa (labele):", y_test.shape)

Dimenzije trening skupa (značajke): (105, 4)
Dimenzije trening skupa (labele): (105, 3)
Dimenzije testnog skupa (značajke): (45, 4)
Dimenzije testnog skupa (labele): (45, 3)


In [4]:
# Hiperparametri neuronske mreže
n_inputs = 4
n_hidden = 20
n_classes = 3
num_samples = X_train.shape[0]

def probs_function(p, X_data):
    """
    Forward pass kroz neuronsku mrežu.
    Argumenti:
        p: Čestica (1D niz) koja sadrži sve težine i otklone.
        X_data: Ulazni podaci (npr. X_train).
    Vraća:
        Matricu vjerovatnoća za svaku klasu.
    """
    # Izdvajanje težina i otklona iz čestice 'p'
    W1 = p[0:80].reshape((n_inputs, n_hidden))
    b1 = p[80:100].reshape((n_hidden,))
    W2 = p[100:160].reshape((n_hidden, n_classes))
    b2 = p[160:163].reshape((n_classes,))

    # Propagacija kroz mrežu
    z1 = X_data.dot(W1) + b1
    a1 = np.tanh(z1)  # Aktivacijska funkcija skrivenog sloja
    logits = a1.dot(W2) + b2

    # Softmax aktivacijska funkcija za izlazni sloj
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    return probs

In [5]:
def forward_prop(params):
    """
    Pomoćna funkcija koja računa gubitak (loss) za jednu česticu.
    """
    # Računanje vjerovatnoća pomoću trening podataka
    probs = probs_function(params, X_train)

    # Korištenje kategoričke unakrsne entropije kao funkcije gubitka
    cce = losses.CategoricalCrossentropy()
    loss = cce(y_train, probs).numpy()

    return loss

def f(x):
    """
    Objektivna funkcija za PSO. Računa gubitak za sve čestice u roju.
    Argument:
        x: Matrica koja sadrži sve čestice u roju (dimenzije: [n_particles, n_dimensions]).
    Vraća:
        Niz gubitaka, po jedan za svaku česticu.
    """
    n_particles = x.shape[0]
    # Izračunaj gubitak za svaku česticu koristeći list comprehension
    values = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(values)

In [6]:
%%time

# PSO hiperparametri
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

# Računanje dimenzije jedne čestice (ukupan broj težina i otklona)
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes

# Kreiranje instance PSO optimizatora
# n_particles=100 znači da će roj imati 100 čestica
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Pokretanje procesa optimizacije
# iters=100 znači da će algoritam proći kroz 100 iteracija
cost, pos = optimizer.optimize(f, iters=100)

print(f"\nNajmanji postignuti trošak (loss): {cost}")
print(f"Broj parametara (dimenzija) u najboljoj čestici: {len(pos)}")

2025-06-13 20:04:57,129 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.106
2025-06-13 20:05:34,872 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.10564613342285156, best pos: [-0.70803776  2.36545153 -0.02117014 -1.11263518  1.65412502 -1.03944382
 -0.18020292  1.08266859 -0.173056    0.3191151  -0.58113787  0.53096837
 -0.11677524 -0.192576    0.08626154  0.57636799  0.10487705  0.31901811
  0.56839214 -0.55235788  0.37366269 -0.92545621  1.45156266 -0.19607175
  0.29438963  0.87689724 -0.20876876 -0.0024108  -0.54209594  0.51719748
  1.17061584  0.90420202 -1.13025871  0.10896095 -0.4948527   0.64612664
  0.06999273  0.31215779 -0.87896002 -0.44421015  0.98997795 -0.27290388
 -0.69930516  1.1673889   0.91829371  1.1911379   1.91624221  0.41858415
  1.59055973  1.26790319  1.53193906  1.68421792  0.76033493  0.39992946
  0.50524277 -0.2737


Najmanji postignuti trošak (loss): 0.10564613342285156
Broj parametara (dimenzija) u najboljoj čestici: 163
CPU times: user 34.1 s, sys: 1.61 s, total: 35.7 s
Wall time: 37.8 s


In [7]:
def predict(position, X_data):
    """
    Koristi najbolju pronađenu česticu (poziciju) za predikciju na novim podacima.
    """
    probs = probs_function(position, X_data)
    return probs

# Izračunavanje tačnosti na testnom skupu
# 1. Dobijamo predikcije (vjerovatnoće) za testni skup
predictions = predict(pos, X_test)
# 2. Zaokružujemo vjerovatnoće da dobijemo one-hot predikcije (npr. [0.1, 0.8, 0.1] -> [0, 1, 0])
rounded_predictions = np.round(predictions).astype('int')
# 3. Poredimo predikcije sa stvarnim labelama i računamo prosječnu tačnost
accuracy = (rounded_predictions == y_test).all(axis=1).mean()

print(f"\nTačnost modela na testnom skupu: {accuracy * 100:.2f}%")


Tačnost modela na testnom skupu: 100.00%
